# NVIDIA AI Blueprint for Video Search and Summarization: Docker Deployment

This notebook will go over the steps to deploy the NVIDIA AI Blueprint for Video Search and Summarization which can ingest massive volumes of live or archived videos and extract insights for summarization and interactive Q&A.

We will go over the docker compose deployment steps which as an alternative to the helm chart deployment. As this blueprint uses multiple models including a VLM, LLM, Reranker, Embedding model, there are multiple ways to deploy the blueprint based on the model deployment (Self-hosted or NV-hosted /w API Key). Here we will be running all models locally/self-hosted.

**Note**: this notebook is designed to run as a [brev.dev launchable](https://console.brev.dev/launchable/deploy/?launchableID=env-2olGFXbhH0qEtU47MJ4tpxCAZIn) on 8XL40S GPU **(Specifically CRUSOE Cloud Provider with Ephemeral storage)**

## Prerequisites

### 1. Obtain NVIDIA API Keys

This key will be used to pull relevant models and containers from build.nvidia.com and NGC.

Generate the key from [NGC Portal](https://ngc.nvidia.com/). Follow the instructions to [Generate NGC API Key.](https://docs.nvidia.com/ngc/gpu-cloud/ngc-user-guide/index.html#generating-api-key)

<div class="alert alert-block alert-success">
    <b>Note:</b>  If you have authentication issues when pulling the NIMs, please verify you have the following <a href="https://org.ngc.nvidia.com/subscriptions" target="_blank">Subscriptions</a>: <strong>NVIDIA Developer Program</strong>
 </div>



In [ ]:
import os

os.environ["NGC_API_KEY"] = "***" #Replace with your key
os.environ["HF_TOKEN"] = "***" #Replace with your huggingface token

### 2. Specify the Vision Language Model (VLM)

Here we are configuring the setup to deploy the default Cosmos Reason 2 VLM. Steps to configure a different VLM can be found here: [Configure the VLM](https://docs.nvidia.com/vss/latest/content/installation-vlms-docker-compose.html#). 

Add environments variables accordingly.

In [ ]:
# Cosmos Reason 2
os.environ["VLM_MODEL_TO_USE"] = "cosmos-reason2"
os.environ["MODEL_PATH"] = "git:https://huggingface.co/nvidia/Cosmos-Reason2-8B"


### 3. Configure Features

#### Computer Vision Pipeline

Set ```DISABLE_CV_PIPELINE``` to ```true``` if you want to disable CV pipeline.

In [ ]:
## CV Pipeline

os.environ["DISABLE_CV_PIPELINE"] = "false" #Set to true to disable
os.environ["NUM_CV_CHUNKS_PER_GPU"] = "1"
os.environ["INSTALL_PROPRIETARY_CODECS"] = "true"

#### Audio Transcription

This uses a ASR NIM, which is deployed in Step 4 of this notebook. To disable audio feature, set the following parameter to ```false```.

In [ ]:
os.environ["ENABLE_AUDIO"] = "true" #Set to false to disable

os.environ["RIVA_ASR_SERVER_URI"] = "host.docker.internal"
os.environ["RIVA_ASR_GRPC_PORT"] = "50051"
os.environ["RIVA_ASR_HTTP_PORT"] = "9005"
os.environ["RIVA_ASR_SERVER_IS_NIM"] = "true"
os.environ["ENABLE_RIVA_SERVER_READINESS_CHECK"] = "true"

---

#### Ensuring user is on right path
Ignore the warning

In [ ]:
%cd /home/ubuntu/video-search-and-summarization/deploy/docker/launchables
!ls

### Verify the Driver and CUDA version to be the following:
- Driver Version: 565.57.01
- CUDA Version 12.7

<div class="alert alert-block alert-success">
     <b>Note:</b> NVIDIA driver version 580+ is needed for Blackwell GPUs. Version 565 is supported for L40S, which is configured for this launchable deployment.
 </div>

In [ ]:
!nvidia-smi

If the driver version doesn't match in the above step, or is giving issues:
- Update the Driver to 580
- Reboot the system
- Set ```NGC_API_KEY``` again

In [ ]:
# !sudo apt install {nvidia-driver,nvidia-kernel-source,nvidia-kernel-common,nvidia-dkms,nvidia-compute-utils,nvidia-utils,libnvidia-gl,libnvidia-compute,libnvidia-decode,libnvidia-cfg1,libnvidia-extra,libnvidia-encode,libnvidia-fbc1,xserver-xorg-video-nvidia}-580=580.65.06-0ubuntu1 -y
# !sudo reboot now

## Deployment: Using all self-hosted models

We will be using Cosmos Nemotron VLM, which is part of the main container. All other models need to be set up before proceeding with the blueprint container. These include:
- [Embedding NIM](https://build.nvidia.com/nvidia/llama-3_2-nv-embedqa-1b-v2)
- [Reranker NIM](https://build.nvidia.com/nvidia/llama-3_2-nv-rerankqa-1b-v2)
- [LLM NIM](https://build.nvidia.com/meta/llama-3_1-70b-instruct)
- [Riva ASR NIM](https://build.nvidia.com/nvidia/parakeet-ctc-0_6b-asr) [Optional, required to enable audio transcription]


### GPU Configuration

<img src="images/vss_gpu_layout_v2.png" width="800">

In order to update the GPUs used by each model:
- **LLM, Embedding and Reranking models:** Update the ```--gpus``` parameters while deploying NIMs in Steps 2-4 below.
- **VLM:** Update ```NVIDIA_VISIBLE_DEVICES``` in docker/.env


### Step 1: Set Environment Variables and Login to Docker

In [ ]:
#os.environ["LOCAL_NIM_CACHE"] = os.path.expanduser("~/.cache/nim") #default cache location
os.environ["LOCAL_NIM_CACHE"] = os.path.expanduser("/ephemeral/cache/nim") #updating with ephemeral storage
os.makedirs(os.environ["LOCAL_NIM_CACHE"], exist_ok=True)

In [ ]:
%%bash
echo "${NGC_API_KEY}" | docker login nvcr.io -u '$oauthtoken' --password-stdin

Updating the docker storage path to Ephemeral storage

In [ ]:
import json, subprocess, time

storage_path = "/ephemeral/cache/docker"

daemon_file = "/etc/docker/daemon.json" #update the path if required
config = {}
try:
    config = json.load(open(daemon_file)) if os.path.exists(daemon_file) else {}
except PermissionError:
    print("Cannot read the file. Try running with elevated privileges or check docker deamon file path.")

config["data-root"] = storage_path
config_str = json.dumps(config, indent=4)

subprocess.run(f"echo '{config_str}' | sudo tee {daemon_file} > /dev/null", shell=True, check=True)
subprocess.run("sudo systemctl restart docker", shell=True, check=True)

time.sleep(5)

# Verify new storage location
print(subprocess.run("docker info | grep 'Docker Root Dir'", shell=True, capture_output=True, text=True).stdout)

### Step 2: Launch the LLM NIM.
Note: If you're logged in as root, make a separate llm_user account and give it permission to the nim cache folder.

Here, we have preset the GPUs to use ```--gpus``` and port ```-p``` for the most optimal deployment on 8xL40s.

In [ ]:
!docker run -it --rm \
    --gpus '"device=0,1,2,3"' \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 8000:8000 \
    -d \
    nvcr.io/nim/meta/llama-3.1-70b-instruct:1.10.1

### Step 3: Launch the Reranker NIM. 

Here we have preset the GPUs to use ```--gpus``` and port ```-p``` for the most optimal deployment on 8xL40s.

In [ ]:
!docker run -it --rm \
    --gpus '"device=4"' \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 9235:8000 \
    -d \
    nvcr.io/nim/nvidia/llama-3.2-nv-rerankqa-1b-v2:1.7.0

### Step 4: Launch the Embedding NIM.

Again, we have preset the GPUs to use ```--gpus``` and port ```-p``` for the most optimal deployment on 8xL40s.

In [ ]:
!docker run -it --rm \
    --gpus '"device=4"' \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 9234:8000 \
    -d \
    nvcr.io/nim/nvidia/llama-3.2-nv-embedqa-1b-v2:1.9.0

### Step 5: Launch the Riva ASR NIM for Audio Support [Optional, only required for audio capabilities]

Now we'll add the Riva ASR NIM for audio capabilities. This enables speech-to-text functionality for video summaries. 

Make sure ```ENABLE_AUDIO``` is set to ```true``` in the prerequisites section.

In [ ]:
!docker network create vss_network

In [ ]:
!docker run -d -it --rm \
    --name parakeet-ctc-asr \
    --network vss_network \
    -p 50051:50051 \
    -p 9005:9000 \
    -e NIM_GRPC_API_PORT=50051 \
    --gpus '"device=5"' \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    nvcr.io/nim/nvidia/parakeet-0-6b-ctc-en-us:2.0.0

### Step 6: Verify all the NIMs are running

After running the following cell, you should be able to see three containers, one for each model.

![Container List](images/containers_pre_check_v2.png)

In [ ]:
!docker ps

Below cell ensures that the LLM NIM is running before proceeding.

<div class="alert alert-block alert-success">
    <b>Note:</b>  LLM NIM service could take a couple of miniutes to get ready.
 </div>

In [ ]:
import requests

url = 'http://localhost:8000/v1/health/ready' #make sure the LLM NIM port is correct
headers = {'accept': 'application/json'}

print("Checking LLM NIM readiness...")
while True:
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get("message") == "Service is ready.":
                print("LLM NIM is ready.")
                break
            else:
                print("LLM NIM is not ready. Waiting for 30 seconds...")
        else:
            print(f"Unexpected status code {response.status_code}. Waiting for 30 seconds...")
    except requests.ConnectionError:
        print("LLM NIM is not ready. Waiting for 30 seconds...")
    time.sleep(30)

In [ ]:
# Check Riva NIM readiness
riva_url = 'http://localhost:9005/v1/health/ready'
headers = {'accept': 'application/json'}

print("Checking Riva ASR NIM readiness...")
while True:
    try:
        response = requests.get(riva_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get("status") == "ready":
                print("Riva ASR NIM is ready!")
                break
            else:
                print("Riva ASR NIM is not ready. Waiting for 30 seconds...")
        else:
            print(f"Unexpected status code {response.status_code}. Waiting for 30 seconds...")
    except requests.ConnectionError:
        print("Riva ASR NIM is not ready. Waiting for 30 seconds...")
    time.sleep(30)

### Step 7: Launch the Blueprint

Before proceeding, make sure you have compose.yaml, config.yaml and .env in the current directory

In [ ]:
!ls -a

#### Update docker compose version (recommended v2.32.4)

In [ ]:
!docker compose version

!mkdir -p ~/.docker/cli-plugins
!curl -SL https://github.com/docker/compose/releases/latest/download/docker-compose-linux-x86_64 -o ~/.docker/cli-plugins/docker-compose
!chmod +x ~/.docker/cli-plugins/docker-compose

In [ ]:
!docker compose version

#### **Docker Compose Deployment**

Here, we start the docker container to spin up the blueprint. The container performs the following main steps:
1. Downloads the VLM
2. Performs model calibration
3. Generates a TRT-LLM Engine
4. Spins up the Milvus and Neo4J database.
5. Starts Video Search and Summarization service
6. Finally, we get the frontend and backend endpoints

<div class="alert alert-block alert-success">
    <b>Note:</b>  This step can take around 30 minutes for the first run as it goes through Steps 1-3 mentioned above
 </div>

The below code cell filters out important logs so that the output is not very verbose

In [ ]:
%%capture
!docker compose down

In [ ]:
import subprocess
import time

keywords = ["Milvus server started", "Downloading model", "Downloaded model", "Initializing Cosmos-Reason1-7B", "Starting to load model", 
            "Starting quantization", "Quantization done", "Engine generation completed", "TRT engines generated", "Uvicorn", 
            "VIA Server loaded", "Backend", "Frontend", "****"]

# Start the docker compose process in detached mode
subprocess.run(['docker', 'compose', 'up', '--quiet-pull', '-d'])

def filter_logs(logs, keywords):
    return [line for line in logs.splitlines() if any(keyword in line for keyword in keywords)]

printed_lines = set()

try:
    while True:
        logs = subprocess.check_output(['docker', 'compose', 'logs', '--no-color'], universal_newlines=True)
        filtered_logs = filter_logs(logs, keywords)
        new_logs = [line for line in filtered_logs if line not in printed_lines]
        
        for line in new_logs:
            print(line)
            printed_lines.add(line)
            if "Frontend" in line:
                print("VSS Server ran successfully.")
                print("Access VSS Frontend UI from Brev portal tunnels section. Refer to Step 7 for more details.")
                raise SystemExit
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping log tailing...")
except SystemExit:
    pass

### Step 8: Access VSS UI using the Secure Links on Brev

1. Go to the "Access" Tab on Brev and scroll down to the "Using Secure Links" section   
    <img src="images/brev_access_tab.png" alt="Access Tab" width="1200"/>

2. (Optional) Add the VSS frontend port "9100" if not already added. This is set in .env file which is configurable   
    <img src="images/brev_add_port.png" alt="Access Tab" width="1200"/>

3. Click on the frontend port (9100) Sharable URL link to access blueprint UI  
    <div class="alert alert-block alert-success">
        <b>Note:</b>  Please reload the brev page if the shareable URL is showing as unhealthy or follow the alternative steps with VSCode below.
    </div>
    <img src="images/brev_vss_ui_9100.png" alt="Access Tab" width="1200"/>

4. Experience VSS using the gradio based UI application. For quick steps to summarize a video, refer to [this link](https://docs.nvidia.com/vss/latest/content/sample_summarization.html). Additionally, for detailed instructions on how to use the UI, follow [this guide](https://docs.nvidia.com/vss/latest/content/ui_app.html)  
    <img src="images/vss_landing_page.png" alt="Access Tab" width="1200"/>

### [Alertnative Option] Access instance on VSCode and add ports

1. First, go to the "Access" tab  
    <img src="images/brev_access_tab.png" alt="Access Tab" width="1200"/>

2. Refer to the commands in "Using Brev CLI" to open VSCode locally  
    <div class="alert alert-block alert-success">
        <b>Note:</b>  Make sure to <a href="https://code.visualstudio.com/docs/setup/mac#_configure-the-path-with-vs-code" target="_blank">Configure the path with VS Code</a> if you run into erros while accessing instance through VSCode.
    </div>  
    <img src="images/brev_vscode_access.png" alt="Access Tab" width="1200"/>

3. Navigate to the Ports view in the Panel region (Ports: Focus on Ports View), and select Forward a Port  
    <img src="images/vscode_ports.png" alt="VSCode Ports" width="1200"/>

4. Add frontend (9100) and backend (8100) ports, and access blueprint UI by clicking on the "localhost:9100"  
    <img src="images/vscode_access_vss_ui.png" alt="VSCode Ports" width="1200"/>

### Uninstalling the blueprint

Uncomment the following cell to stop the blueprint container, followed by stopping all model containers.

In [ ]:
# %%capture
# # To bring down the blueprint instance
# !docker compose down

# # To stop all other containers
# !docker stop $(docker ps -q)

In [ ]:
!docker ps